# Cross Validation

1. Reservar 1/k de los datos como muestra reservada.
2. Entrenar el modelo con los datos restantes
3. Aplicar (puntuar) el modelo a la retención de 1/k y registrar las métricas de evaluación del modelo.
4. Restaurar el primer 1/k de los datos y reservar el siguiente 1/k (excluyendo los registros que se seleccionaron la primera vez)
5. Repetir los pasos 2 y 3
6. Repetir hasta que cada registro se haya utilizado en la parte reservada
7. Promediar o combinar las métricas de evaluación del modelo.

In [3]:
# Reto: Seleccionar las caracteristicas que creen que son más importantes y hacer el proceso de regresión lineal y obtener métricas e interpretar.

In [15]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

from dmba.featureSelection import stepwise_selection

from dmba.metric import AIC_score

no display found. Using non-interactive Agg backend


In [11]:
csv = pd.read_csv("../Datasets/house_sales.csv",delimiter="\t")

In [6]:
predictors = ['SqFtLot','Bedrooms','SqFtFinBasement','Bathrooms','NewConstruction']
outcome='AdjSalePrice'

In [7]:
data[predictors]

,SqFtLot,Bedrooms,SqFtFinBasement,Bathrooms,NewConstruction
1,9373,6,0,3.00,False
2,20156,4,1452,3.75,True
3,26036,4,900,1.75,False
4,8618,5,1640,3.75,False
5,8620,4,0,1.75,False
...,...,...,...,...,...
27057,5468,3,590,1.75,False
27058,23914,4,910,4.50,False
27061,11170,4,0,1.00,False
27062,6223,3,0,2.00,False


In [8]:
data = pd.get_dummies(data[predictors],drop_first=True)
data

,SqFtLot,Bedrooms,SqFtFinBasement,Bathrooms,NewConstruction
1,9373,6,0,3.00,False
2,20156,4,1452,3.75,True
3,26036,4,900,1.75,False
4,8618,5,1640,3.75,False
5,8620,4,0,1.75,False
...,...,...,...,...,...
27057,5468,3,590,1.75,False
27058,23914,4,910,4.50,False
27061,11170,4,0,1.00,False
27062,6223,3,0,2.00,False


In [9]:
data['NewConstruction'] = [1 if d else 0 for d in data['NewConstruction']]
data

,SqFtLot,Bedrooms,SqFtFinBasement,Bathrooms,NewConstruction
1,9373,6,0,3.00,0
2,20156,4,1452,3.75,1
3,26036,4,900,1.75,0
4,8618,5,1640,3.75,0
5,8620,4,0,1.75,0
...,...,...,...,...,...
27057,5468,3,590,1.75,0
27058,23914,4,910,4.50,0
27061,11170,4,0,1.00,0
27062,6223,3,0,2.00,0


In [12]:
# Entrenamiento con los datos
model = sm.OLS(csv[outcome],data.assign(const=1))
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           AdjSalePrice   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     2063.
Date:                Wed, 04 May 2022   Prob (F-statistic):               0.00
Time:                        18:22:17   Log-Likelihood:            -3.1974e+05
No. Observations:               22687   AIC:                         6.395e+05
Df Residuals:                   22681   BIC:                         6.395e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
SqFtLot             1.0293      0.074     13.936      0.000       0.885       1.174
Bedrooms        -1790.1275   2852.539     -0.628      0.530   -7381.299    3801.044
SqFtFinBasement   140.5929      5.192     27.079      0.000     130.416     150.769
Bathrooms        2.436e+05   3428.979     71.047      0.000    2.37e+05     2.5e+05
NewConstruction  -3.92e+04   7211.965     -5.435      0.000   -5.33e+04   -2.51e+04
const           -8159.1692   8558.826     -0.953      0.340   -2.49e+04    8616.717
==============================================================================
Omnibus:                    25897.944   Durbin-Watson:                   1.205
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7339792.177
Skew:                           5.561   Prob(JB):                         0.00
Kurtosis:                      90.412   Cond. No.                     1.30e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.3e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:

def train(variables):
    if len(variables)==0:
        return None
    model = LinearRegression()
    model.fit(csv[variables],csv[outcome])
    return model

def score_model(model,variables):
    if len(variables)==0:
        return None # Como obtener el AIC_score cuando no hay variables
    return AIC_score(csv[outcome],model=model.predict(csv[variables],model))

best = stepwise_selection(csv.columns,train_model=train,score_model=score_model,verbose=True)

Variables: DocumentDate, SalePrice, PropertyID, PropertyType, ym, zhvi_px, zhvi_idx, AdjSalePrice, NbrLivingUnits, SqFtLot, SqFtTotLiving, SqFtFinBasement, Bathrooms, Bedrooms, BldgGrade, YrBuilt, YrRenovated, TrafficNoise, LandVal, ImpsVal, ZipCode, NewConstruction


TypeError: unsupported format string passed to NoneType.__format__